In [28]:
import os, json, time, random
from pathlib import Path

import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

import openslide
import cv2
from PIL import Image
from skimage import morphology

import h5py

import torch
import torch.nn as nn
import torch.nn.functional as F

In [29]:
labels_path = "multiomics_labels.tsv"
labels = pd.read_csv(labels_path, sep="\t")

if "patient_id" in labels.columns:
    patient_col = "patient_id"
elif "submitter_id" in labels.columns:
    patient_col = "submitter_id"
else:
    raise ValueError(f"Cannot find patient id column. Columns: {labels.columns.tolist()}")

# Label column
if "subtype_simple" in labels.columns:
    label_col = "subtype_simple"
elif "label" in labels.columns:
    label_col = "label"
else:
    raise ValueError(f"Cannot find label column. Columns: {labels.columns.tolist()}")

labels[patient_col] = labels[patient_col].astype(str).str.strip()
labels[label_col] = labels[label_col].astype(str).str.strip()

# keep only LUAD/LUSC rows
labels = labels[labels[label_col].isin(["LUAD", "LUSC"])].copy().reset_index(drop=True)

cases = labels[patient_col].tolist()

print("patient_col:", patient_col)
print("label_col:", label_col)
print("Total cases:", len(cases))
print(labels[label_col].value_counts())
print("First 5 cases:", cases[:5])

patient_col: patient_id
label_col: subtype_simple
Total cases: 831
subtype_simple
LUAD    459
LUSC    372
Name: count, dtype: int64
First 5 cases: ['TCGA-MP-A4SV', 'TCGA-55-8621', 'TCGA-MN-A4N1', 'TCGA-55-6986', 'TCGA-86-6851']


GDC: Find svs for one case

In [30]:
GDC_BASE = "https://api.gdc.cancer.gov"

def find_svs_slide_for_case(case_id, prefer="DX"):
    filters_obj = {
        "op": "and",
        "content": [
            {"op": "in", "content": {"field": "cases.submitter_id", "value": [case_id]}},
            {"op": "in", "content": {"field": "files.data_type", "value": ["Slide Image"]}},
            {"op": "in", "content": {"field": "files.data_format", "value": ["SVS"]}},
        ]
    }
    params = {
        "filters": json.dumps(filters_obj),
        "fields": "file_id,file_name,file_size,access,cases.submitter_id",
        "format": "JSON",
        "size": 200
    }
    r = requests.get(f"{GDC_BASE}/files", params=params, timeout=60)
    r.raise_for_status()
    hits = r.json().get("data", {}).get("hits", [])
    if not hits:
        return None

    # prefer DX
    for h in hits:
        if prefer in h.get("file_name", ""):
            return h
    return hits[0]

# quick test
test_case = cases[0]
hit = find_svs_slide_for_case(test_case)
print("Test case:", test_case)
print("Hit:", hit)

Test case: TCGA-MP-A4SV
Hit: {'id': 'f7013b7b-9227-4606-9594-54689b8ab13b', 'cases': [{'submitter_id': 'TCGA-MP-A4SV'}], 'access': 'open', 'file_name': 'TCGA-MP-A4SV-01Z-00-DX1.5430F2CF-BE4B-42D6-9CD4-92E6FB3F6E99.svs', 'file_id': 'f7013b7b-9227-4606-9594-54689b8ab13b', 'file_size': 1386511092}


Download one svs

In [31]:
temp_dir = Path("./temp_slides")
temp_dir.mkdir(exist_ok=True, parents=True)

def download_svs(file_id, out_path=None, retries=3):
    if out_path is None:
        out_path = temp_dir / f"{file_id}.svs"
    url = f"{GDC_BASE}/data/{file_id}"

    for attempt in range(1, retries + 1):
        try:
            r = requests.get(url, stream=True, timeout=60)
            r.raise_for_status()
            total = int(r.headers.get("content-length", 0))

            with open(out_path, "wb") as f:
                if total > 0:
                    with tqdm(total=total, unit="B", unit_scale=True) as pbar:
                        for chunk in r.iter_content(chunk_size=1024 * 1024):
                            if chunk:
                                f.write(chunk)
                                pbar.update(len(chunk))
                else:
                    for chunk in r.iter_content(chunk_size=1024 * 1024):
                        if chunk:
                            f.write(chunk)

            return out_path

        except Exception as e:
            print(f"Download attempt {attempt} failed:", e)
            if out_path.exists():
                out_path.unlink()
            time.sleep(2 * attempt)

    return None

svs_path = download_svs(hit["file_id"])
print("Downloaded:", svs_path)

100%|██████████| 1.39G/1.39G [01:12<00:00, 19.3MB/s]

Downloaded: temp_slides/f7013b7b-9227-4606-9594-54689b8ab13b.svs


In [32]:
slide = openslide.OpenSlide(str(svs_path))
print("Level count:", slide.level_count)
print("Level dimensions:", slide.level_dimensions)
print("Level downsamples:", slide.level_downsamples)
slide.close()

Level count: 4
Level dimensions: ((71747, 85607), (17936, 21401), (4484, 5350), (2242, 2675))
Level downsamples: (1.0, 4.000153720869589, 16.000988728355026, 32.00197745671005)


In [33]:
def tissue_mask_hsv(slide, mask_level=3):
    mask_level = min(mask_level, slide.level_count - 1)
    w, h = slide.level_dimensions[mask_level]
    thumb = slide.read_region((0, 0), mask_level, (w, h)).convert("RGB")
    img = np.array(thumb)

    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    S = hsv[:, :, 1]
    V = hsv[:, :, 2]

    tissue = (S > 20) & (V < 245)

    # cleanup
    tissue = morphology.remove_small_objects(tissue, min_size=500)
    tissue = morphology.remove_small_holes(tissue, area_threshold=500)
    tissue = morphology.closing(tissue, morphology.disk(3))

    return tissue.astype(np.uint8), mask_level

slide = openslide.OpenSlide(str(svs_path))
mask_low, used_level = tissue_mask_hsv(slide, mask_level=3)
print("Mask level used:", used_level)
print("Mask shape:", mask_low.shape, "tissue fraction:", mask_low.mean())
slide.close()

Mask level used: 3
Mask shape: (2675, 2242) tissue fraction: 0.6290018091323668


Sample patch coordinates

In [34]:
PATCH_SIZE = 224
EXTRACT_LEVEL = 0
MIN_TISSUE = 0.60
TARGET_PATCHES = 2000
STEP = 224  # start fast; later we can reduce (e.g., 112) to get more candidates

def sample_patch_coords(mask_scaled, patch_size=224, min_tissue=0.6, n_patches=2000, step=224):
    H, W = mask_scaled.shape
    candidates = []
    for y in range(0, H - patch_size + 1, step):
        for x in range(0, W - patch_size + 1, step):
            if mask_scaled[y:y+patch_size, x:x+patch_size].mean() >= min_tissue:
                candidates.append((x, y))
    if len(candidates) <= n_patches:
        return candidates
    return random.sample(candidates, n_patches)

slide = openslide.OpenSlide(str(svs_path))
mask_down = float(slide.level_downsamples[used_level])
ext_down  = float(slide.level_downsamples[EXTRACT_LEVEL])
scale = mask_down / ext_down

mh, mw = mask_low.shape
mask_scaled = cv2.resize(mask_low, (int(mw * scale), int(mh * scale)), interpolation=cv2.INTER_NEAREST)

coords = sample_patch_coords(mask_scaled, PATCH_SIZE, MIN_TISSUE, TARGET_PATCHES, step=STEP)
print("Candidate coords sampled:", len(coords))
slide.close()

Candidate coords sampled: 2000


Extract patches + quality fiter

In [35]:
def quality_ok(patch_rgb, blur_thr=80.0, bright_lo=20, bright_hi=235):
    gray = cv2.cvtColor(patch_rgb, cv2.COLOR_RGB2GRAY)
    if cv2.Laplacian(gray, cv2.CV_64F).var() < blur_thr:
        return False
    m = gray.mean()
    if m < bright_lo or m > bright_hi:
        return False
    return True

slide = openslide.OpenSlide(str(svs_path))

patches = []
coords_level0 = []

for (x, y) in coords:
    x0 = int(x * ext_down)
    y0 = int(y * ext_down)
    patch = slide.read_region((x0, y0), EXTRACT_LEVEL, (PATCH_SIZE, PATCH_SIZE)).convert("RGB")
    if quality_ok(np.array(patch)):
        patches.append(patch)
        coords_level0.append((x0, y0))

slide.close()

print("Patches kept:", len(patches))
print("Example coord:", coords_level0[0] if coords_level0 else None)

Patches kept: 1987
Example coord: (15904, 9408)


Install and login to Hugging Face

In [36]:
from huggingface_hub import login
login()

Load UNI encoder from Hugging Face

In [37]:
import timm, torch
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

UNI_MODEL_ID = "hf-hub:MahmoodLab/UNI"  

encoder = timm.create_model(
    UNI_MODEL_ID,
    pretrained=True,
    num_classes=0,
    init_values=1e-5,     
).to(device).eval()

cfg = resolve_data_config({}, model=encoder)
tfm = create_transform(**cfg, is_training=False)

print("Loaded UNI encoder", encoder.default_cfg.get("architecture", ""))

Device: cuda
Loaded UNI encoder vit_large_patch16_224


UNI embedder

In [38]:
@torch.no_grad()
def uni_embed_batch(pil_tiles, encoder, tfm, device, batch_size=64):
    encoder.eval()
    out = []
    for i in range(0, len(pil_tiles), batch_size):
        batch = torch.stack([tfm(t) for t in pil_tiles[i:i+batch_size]]).to(device, non_blocking=True)
        feats = encoder(batch)  # (B, 1024)
        out.append(feats.detach().cpu().numpy())
    return np.concatenate(out, axis=0).astype(np.float32)

Read patches from slide using coords

In [39]:
def read_patches_from_coords(slide, coords, tile_size=224, level=0):
    tiles = []
    for (x, y) in coords:
        tile = slide.read_region((int(x), int(y)), level, (tile_size, tile_size)).convert("RGB")
        tiles.append(tile)
    return tiles

In [40]:
def extract_uni_to_h5(wsi_path, coords, out_h5_path, encoder, tfm, device,
                      tile_size=224, level=0, batch_size=64):
    slide = openslide.OpenSlide(wsi_path)

    coords_np = np.array(coords, dtype=np.int32)
    n = coords_np.shape[0]

    with h5py.File(out_h5_path, "w") as f:
        d_feat = f.create_dataset("features", shape=(n, 1024), dtype=np.float32, chunks=True)
        d_xy   = f.create_dataset("coords", data=coords_np, dtype=np.int32)

        for i in tqdm(range(0, n, batch_size), desc=os.path.basename(wsi_path)):
            batch_coords = coords_np[i:i+batch_size]
            tiles = read_patches_from_coords(slide, batch_coords, tile_size=tile_size, level=level)
            feats = uni_embed_batch(tiles, encoder, tfm, device=device, batch_size=len(tiles))
            d_feat[i:i+feats.shape[0]] = feats

    slide.close()
    return out_h5_path

In [41]:
# USE THE COORDS YOU ALREADY COMPUTED (in memory)

wsi_path = str(svs_path)                  # downloaded slide path
coords = np.array(coords_level0)          # from your patch sampling step

case_id = test_case                       # e.g. "TCGA-MP-A4SV"

out_dir = Path("bags")
out_dir.mkdir(exist_ok=True)

out_h5 = out_dir / f"{case_id}.h5"

extract_uni_to_h5(
    wsi_path=wsi_path,
    coords=coords,
    out_h5_path=str(out_h5),
    encoder=encoder,
    tfm=tfm,
    device=device,
    tile_size=224,
    level=0,
    batch_size=64
)

print("Saved:", out_h5)

f7013b7b-9227-4606-9594-54689b8ab13b.svs: 100%|██████████| 32/32 [00:46<00:00,  1.44s/it]

Saved: bags/TCGA-MP-A4SV.h5


In [42]:
import h5py

h5_path = "bags/TCGA-MP-A4SV.h5"
with h5py.File(h5_path, "r") as f:
    print("Keys:", list(f.keys()))
    for k in f.keys():
        print(k, f[k].shape, f[k].dtype)

Keys: ['coords', 'features']
coords (1987, 2) int32
features (1987, 1024) float32


In [43]:
from pathlib import Path

try:
    Path(svs_path).unlink()
    print("Deleted SVS:", svs_path)
except Exception as e:
    print("Warning: could not delete SVS:", e)

Deleted SVS: temp_slides/f7013b7b-9227-4606-9594-54689b8ab13b.svs


In [44]:
from pathlib import Path
import traceback

bags_dir = Path("bags")
bags_dir.mkdir(exist_ok=True)

processed_log = Path("processed_cases.txt")
error_log = Path("error_cases.txt")

def log_line(path, line):
    with open(path, "a") as f:
        f.write(line + "\n")

def get_processed_set():
    if processed_log.exists():
        return set(x.strip() for x in processed_log.read_text().splitlines() if x.strip())
    return set()

def process_one_case(case_id, prefer="DX"):
    try:
        hit = find_svs_slide_for_case(case_id, prefer=prefer)
        if hit is None:
            log_line(error_log, f"{case_id}\tNO_SVS_FOUND")
            return False

        # download
        svs_path_local = download_svs(hit["file_id"])
        if svs_path_local is None or not Path(svs_path_local).exists():
            log_line(error_log, f"{case_id}\tDOWNLOAD_FAILED\t{hit['file_id']}")
            return False

        # coords + H5 output
        # (this assumes we already have our patch mask+sampling code packaged;
        # if not yet, we'll plug it in next cell)
        return True

    except Exception as e:
        log_line(error_log, f"{case_id}\tEXCEPTION\t{repr(e)}")
        return False

In [45]:
import numpy as np
import openslide
import cv2
from pathlib import Path
import traceback

bags_dir = Path("bags")
bags_dir.mkdir(exist_ok=True)

processed_log = Path("processed_cases.txt")
error_log = Path("error_cases.txt")

PATCH_SIZE = 224
EXTRACT_LEVEL = 0
MASK_LEVEL = 3
MIN_TISSUE = 0.60
TARGET_PATCHES = 2000
STEP = 224  # change to 112 if you want more candidates

def log_line(path, line):
    with open(path, "a") as f:
        f.write(line + "\n")

def get_processed_set():
    if processed_log.exists():
        return set(x.strip() for x in processed_log.read_text().splitlines() if x.strip())
    return set()

def build_coords_from_hsv(wsi_path):
    slide = openslide.OpenSlide(str(wsi_path))

    # low-res mask
    mask_low, used_level = tissue_mask_hsv(slide, mask_level=MASK_LEVEL)

    # scale mask to extract level grid
    mask_down = float(slide.level_downsamples[used_level])
    ext_down  = float(slide.level_downsamples[EXTRACT_LEVEL])
    scale = mask_down / ext_down

    mh, mw = mask_low.shape
    mask_scaled = cv2.resize(mask_low, (int(mw * scale), int(mh * scale)), interpolation=cv2.INTER_NEAREST)

    coords = sample_patch_coords(
        mask_scaled,
        patch_size=PATCH_SIZE,
        min_tissue=MIN_TISSUE,
        n_patches=TARGET_PATCHES,
        step=STEP
    )

    # convert to level-0 coords
    coords_level0 = []
    for (x, y) in coords:
        x0 = int(x * ext_down)
        y0 = int(y * ext_down)
        coords_level0.append((x0, y0))

    slide.close()
    return np.array(coords_level0, dtype=np.int64)

def process_one_case(case_id, label, prefer="DX", delete_svs=True):
    out_h5 = bags_dir / f"{case_id}.h5"
    if out_h5.exists():
        log_line(processed_log, case_id)
        return True

    try:
        hit = find_svs_slide_for_case(case_id, prefer=prefer)
        if hit is None:
            log_line(error_log, f"{case_id}\tNO_SVS_FOUND")
            return False
        if hit.get("access") != "open":
            log_line(error_log, f"{case_id}\tNOT_OPEN_ACCESS\t{hit.get('access')}")
            return False

        svs_path_local = download_svs(hit["file_id"])
        if svs_path_local is None or not Path(svs_path_local).exists():
            log_line(error_log, f"{case_id}\tDOWNLOAD_FAILED\t{hit['file_id']}")
            return False

        # coords from HSV
        coords_level0 = build_coords_from_hsv(svs_path_local)
        if len(coords_level0) == 0:
            log_line(error_log, f"{case_id}\tNO_COORDS_AFTER_MASK")
            return False

        # UNI → H5
        extract_uni_to_h5(
            wsi_path=str(svs_path_local),
            coords=coords_level0,
            out_h5_path=str(out_h5),
            encoder=encoder,
            tfm=tfm,
            device=device,
            tile_size=224,
            level=0,
            batch_size=64
        )

        # add metadata
        import h5py
        with h5py.File(out_h5, "a") as f:
            f.attrs["case_id"] = case_id
            f.attrs["label"] = label
            f.attrs["file_id"] = hit["file_id"]
            f.attrs["file_name"] = hit.get("file_name","")
            f.attrs["access"] = hit.get("access","")
            f.attrs["encoder"] = UNI_MODEL_ID
            f.attrs["target_patches"] = int(TARGET_PATCHES)

        log_line(processed_log, case_id)

        return True

    except Exception as e:
        log_line(error_log, f"{case_id}\tEXCEPTION\t{repr(e)}")
        return False

    finally:
        if delete_svs:
            try:
                if "svs_path_local" in locals() and svs_path_local and Path(svs_path_local).exists():
                    Path(svs_path_local).unlink()
            except Exception:
                pass

In [46]:
processed = get_processed_set()

# build a list of (case, label) from your labels df
case_label_pairs = list(zip(labels[patient_col].tolist(), labels[label_col].tolist()))

to_run = [(c, y) for (c, y) in case_label_pairs if c not in processed]
to_run_100 = to_run[:100]

print("Already processed:", len(processed))
print("Running now:", len(to_run_100))

ok = 0
bad = 0

for i, (case_id, lab) in enumerate(to_run_100, 1):
    success = process_one_case(case_id, lab, prefer="DX", delete_svs=True)
    if success:
        ok += 1
    else:
        bad += 1

    if i % 10 == 0:
        print(f"[{i}/{len(to_run_100)}] OK={ok} FAIL={bad}")

print("Done. OK=", ok, "FAIL=", bad)

Already processed: 216
Running now: 100


100%|██████████| 12.1M/12.1M [00:00<00:00, 18.0MB/s]
100%|██████████| 1.20G/1.20G [01:02<00:00, 19.2MB/s]
8f7cdbca-32a7-4bd3-be11-6cf7b7b03d13.svs: 100%|██████████| 32/32 [00:45<00:00,  1.42s/it]
100%|██████████| 928M/928M [00:48<00:00, 19.1MB/s] 
9c6b3ad0-91d9-4a64-acc5-f151d271e275.svs: 100%|██████████| 32/32 [00:40<00:00,  1.28s/it]
100%|██████████| 380M/380M [00:19<00:00, 19.1MB/s] 
1463fdb7-aab6-4c05-a124-00851fe51bf3.svs: 100%|██████████| 32/32 [00:28<00:00,  1.12it/s]
100%|██████████| 681M/681M [00:35<00:00, 19.2MB/s] 
68ac5f4c-ef80-4b31-8bb6-a2492fe9cd58.svs: 100%|██████████| 32/32 [00:28<00:00,  1.12it/s]
100%|██████████| 1.43G/1.43G [01:14<00:00, 19.2MB/s]
1616d5de-a80a-47e8-9ad6-1e22097c5633.svs: 100%|██████████| 32/32 [00:28<00:00,  1.13it/s]
100%|██████████| 1.95G/1.95G [01:41<00:00, 19.2MB/s]
915dfbfe-2bf1-4da0-9cab-05756b54973b.svs: 100%|██████████| 32/32 [00:28<00:00,  1.13it/s]
100%|██████████| 531M/531M [00:27<00:00, 19.2MB/s] 
3f9a8643-6f8c-4c37-a7a4-892e32caf9be.svs

[10/100] OK=9 FAIL=1


100%|██████████| 1.71G/1.71G [01:29<00:00, 19.2MB/s]
fa83283f-77e3-434b-9f40-6965ca43f17f.svs: 100%|██████████| 32/32 [00:28<00:00,  1.14it/s]
100%|██████████| 402M/402M [00:21<00:00, 18.6MB/s] 
19772eb2-49e3-4271-91b6-cdc1b688fa2a.svs: 100%|██████████| 32/32 [00:45<00:00,  1.43s/it]
100%|██████████| 112M/112M [00:06<00:00, 18.4MB/s] 
d4824e95-b846-4829-b47a-f1ededd09406.svs: 100%|██████████| 32/32 [00:42<00:00,  1.34s/it]
100%|██████████| 665M/665M [00:35<00:00, 18.8MB/s] 
2e362dd3-9cab-4074-a546-5086c706b0e2.svs: 100%|██████████| 32/32 [00:28<00:00,  1.13it/s]
100%|██████████| 305M/305M [00:15<00:00, 19.2MB/s] 
cba0303e-5f68-433c-8459-a3565bc7ccab.svs: 100%|██████████| 32/32 [00:36<00:00,  1.13s/it]
100%|██████████| 723M/723M [00:37<00:00, 19.2MB/s] 
b53d1202-bb6f-4494-9e01-4684e09486ba.svs: 100%|██████████| 32/32 [00:37<00:00,  1.16s/it]
100%|██████████| 157M/157M [00:08<00:00, 19.2MB/s] 
1303284b-7746-401d-9970-8e4e2422d256.svs: 100%|██████████| 32/32 [00:42<00:00,  1.32s/it]
100%|

[20/100] OK=19 FAIL=1


...

[80/100] OK=79 FAIL=1


100%|██████████| 392M/392M [00:20<00:00, 19.1MB/s] 
dc0d2b70-5bba-41dd-b33b-1d223ae02136.svs: 100%|██████████| 32/32 [00:44<00:00,  1.39s/it]
100%|██████████| 420M/420M [00:21<00:00, 19.2MB/s] 
c9e28a52-98a1-40f0-91c8-c8ea09a87a36.svs: 100%|██████████| 32/32 [00:37<00:00,  1.16s/it]
100%|██████████| 498M/498M [00:25<00:00, 19.2MB/s] 
23fb8d44-cb75-41c0-b1a0-80158e1096a0.svs: 100%|██████████| 32/32 [00:28<00:00,  1.14it/s]
100%|██████████| 354M/354M [00:18<00:00, 19.1MB/s] 
e539b0b1-95bf-475a-8a59-271826d424cb.svs: 100%|██████████| 32/32 [00:40<00:00,  1.26s/it]
100%|██████████| 136M/136M [00:07<00:00, 19.0MB/s] 
a21ba7ca-1bbb-4b84-9b6e-1d0de4ee28bc.svs: 100%|██████████| 32/32 [00:42<00:00,  1.32s/it]
100%|██████████| 909M/909M [00:47<00:00, 19.2MB/s] 
ed84f981-7ee0-4796-9519-f9e7b9d11291.svs: 100%|██████████| 32/32 [00:43<00:00,  1.36s/it]
100%|██████████| 193M/193M [00:10<00:00, 19.1MB/s] 
94dfeae0-aaa6-405f-bf37-1c17d207f0ca.svs: 100%|██████████| 32/32 [00:35<00:00,  1.12s/it]
100%|█

[90/100] OK=89 FAIL=1


100%|██████████| 431M/431M [00:22<00:00, 19.0MB/s] 
1669aee8-800f-4908-962b-b9cf4211f334.svs: 100%|██████████| 32/32 [00:40<00:00,  1.26s/it]
100%|██████████| 163M/163M [00:08<00:00, 19.1MB/s] 
74e02518-3b74-4ba6-b4a5-044e44f8f370.svs: 100%|██████████| 32/32 [00:43<00:00,  1.35s/it]
100%|██████████| 194M/194M [00:10<00:00, 18.9MB/s] 
7c4036ba-3945-49de-b730-2f252a1d9808.svs: 100%|██████████| 32/32 [00:45<00:00,  1.42s/it]
100%|██████████| 688M/688M [00:35<00:00, 19.2MB/s] 
21c1d40c-ee8e-4c9a-8d30-dab0faeb3fda.svs: 100%|██████████| 32/32 [00:40<00:00,  1.27s/it]
100%|██████████| 1.69G/1.69G [01:28<00:00, 19.1MB/s]
b52b6590-09bd-48e1-9533-7258a6195533.svs: 100%|██████████| 32/32 [00:28<00:00,  1.12it/s]
100%|██████████| 614M/614M [00:31<00:00, 19.2MB/s] 
d3519b7a-b09e-4900-a0ba-dbd44909a43f.svs: 100%|██████████| 32/32 [00:36<00:00,  1.15s/it]
100%|██████████| 239M/239M [00:12<00:00, 19.1MB/s] 
46fe9cc7-73be-4251-8542-6675bd03a21f.svs: 100%|██████████| 32/32 [00:41<00:00,  1.30s/it]
100%|

[100/100] OK=99 FAIL=1
Done. OK= 99 FAIL= 1


In [47]:
import os

bags = os.listdir("bags")
print("Total H5 files:", len(bags))
print("Example file:", bags[0])


Total H5 files: 315
Example file: TCGA-MP-A4SV.h5


In [48]:
import h5py

with h5py.File("bags/" + bags[0], "r") as f:
    print(list(f.keys()))
    print("Features shape:", f["features"].shape)

['coords', 'features']
Features shape: (1987, 1024)


In [49]:
from pathlib import Path

bags_dir = Path("bags")
done_cases = set(p.stem for p in bags_dir.glob("*.h5"))  # stem = filename without .h5

print("Already have H5 for:", len(done_cases), "cases")
print("Example done cases:", list(done_cases)[:5])


Already have H5 for: 315 cases
Example done cases: ['TCGA-64-1681', 'TCGA-78-7633', 'TCGA-55-7816', 'TCGA-L9-A443', 'TCGA-91-A4BD']


In [50]:
from pathlib import Path

bags_dir = Path("bags")
done_cases = sorted(p.stem for p in bags_dir.glob("*.h5"))

checkpoint_file = Path("processed_cases.txt")
checkpoint_file.write_text("\n".join(done_cases))

print("Saved checkpoint:", checkpoint_file)
print("Cases saved:", len(done_cases))

Saved checkpoint: processed_cases.txt
Cases saved: 315


In [51]:
from pathlib import Path

done_cases = set(Path("processed_cases.txt").read_text().splitlines())
print("Loaded done cases:", len(done_cases))


Loaded done cases: 315


In [52]:
done_cases = set(p.stem for p in Path("bags").glob("*.h5"))

In [53]:
done_cases = {p.stem for p in Path("bags").glob("*.h5")}

In [54]:
remaining = [c for c in cases if c not in done_cases]
print("Remaining:", len(remaining))

Remaining: 516


In [55]:
remaining = [c for c in cases if c not in done_cases]
print("Remaining to process:", len(remaining))
print("First 5 remaining:", remaining[:5])

ok = 0
bad = 0

for i, case_id in enumerate(remaining, 1):
    label = labels.loc[labels[patient_col] == case_id, label_col].iloc[0]
    success = process_one_case(case_id, label, prefer="DX", delete_svs=True)

    if success:
        ok += 1
    else:
        bad += 1

    if i % 10 == 0:
        print(f"[{i}/{len(remaining)}] OK={ok} FAIL={bad}")

print("DONE remaining. OK=", ok, "FAIL=", bad)


Remaining to process: 516
First 5 remaining: ['TCGA-05-5420', 'TCGA-55-7227', 'TCGA-55-5899', 'TCGA-55-A48Y', 'TCGA-97-8176']


100%|██████████| 12.1M/12.1M [00:00<00:00, 17.5MB/s]
100%|██████████| 242M/242M [00:12<00:00, 19.2MB/s] 
8a699e69-5f01-4fd9-bf8f-9c85cbdbfd9a.svs: 100%|██████████| 32/32 [00:42<00:00,  1.33s/it]
100%|██████████| 114M/114M [00:06<00:00, 19.0MB/s] 
199e255a-bf01-43c9-a6aa-fa6ff8edf252.svs: 100%|██████████| 32/32 [00:40<00:00,  1.28s/it]
100%|██████████| 101M/101M [00:05<00:00, 19.0MB/s] 
20fc6d32-fa0d-4bf2-b43b-402c78d0564c.svs: 100%|██████████| 32/32 [00:42<00:00,  1.34s/it]
100%|██████████| 94.3M/94.3M [00:04<00:00, 18.9MB/s]
43c27fd0-5829-4f9f-99d6-2a91ea689f8d.svs: 100%|██████████| 32/32 [00:29<00:00,  1.08it/s]
100%|██████████| 250M/250M [00:13<00:00, 19.1MB/s] 
075ccdbe-9d24-44b5-b8a6-2389fa7d683b.svs: 100%|██████████| 32/32 [00:42<00:00,  1.32s/it]
100%|██████████| 1.42G/1.42G [01:14<00:00, 19.2MB/s]
7bc387ff-930d-49d4-b93c-b1cc61d3ca65.svs: 100%|██████████| 32/32 [00:28<00:00,  1.13it/s]
100%|██████████| 297M/297M [00:15<00:00, 19.1MB/s] 
c8f6bd03-0a77-4c93-8caf-fef4b1569085.svs:

[10/516] OK=9 FAIL=1


100%|██████████| 189M/189M [00:09<00:00, 19.1MB/s] 
2e484996-d335-48c5-9e3c-b9380d865ce2.svs: 100%|██████████| 32/32 [00:40<00:00,  1.28s/it]
100%|██████████| 1.26G/1.26G [01:05<00:00, 19.1MB/s]
fa977180-f475-4984-9900-9e181aed27e5.svs: 100%|██████████| 32/32 [00:28<00:00,  1.13it/s]
100%|██████████| 1.19G/1.19G [01:02<00:00, 19.0MB/s]
7902dc56-411c-49cf-a49e-a6c844ea76f8.svs: 100%|██████████| 32/32 [00:43<00:00,  1.36s/it]
100%|██████████| 1.55G/1.55G [01:20<00:00, 19.2MB/s]
85924b95-bea6-4f1d-bd5b-afd67834dbc8.svs: 100%|██████████| 32/32 [00:28<00:00,  1.13it/s]
100%|██████████| 1.68G/1.68G [01:27<00:00, 19.2MB/s]
8fe88cde-3588-46ab-9a0c-3a3fda424f4a.svs: 100%|██████████| 32/32 [00:46<00:00,  1.44s/it]
100%|██████████| 746M/746M [00:38<00:00, 19.2MB/s] 
5d2fec3e-66da-41d7-b90c-c96978efb731.svs: 100%|██████████| 32/32 [00:39<00:00,  1.23s/it]
100%|██████████| 303M/303M [00:15<00:00, 19.1MB/s] 
f608c743-53ba-4223-a800-8e885ba394b4.svs: 100%|██████████| 32/32 [00:28<00:00,  1.13it/s]
10

[20/516] OK=19 FAIL=1


...

[500/516] OK=499 FAIL=1


100%|██████████| 402M/402M [00:21<00:00, 19.1MB/s] 
5048c79f-ed1d-4534-8da2-1a4d207b2371.svs: 100%|██████████| 32/32 [00:37<00:00,  1.19s/it]
100%|██████████| 222M/222M [00:11<00:00, 19.1MB/s] 
0a722c1d-82aa-43ec-abf3-2c4ad09984c9.svs: 100%|██████████| 32/32 [00:29<00:00,  1.09it/s]
100%|██████████| 332M/332M [00:17<00:00, 19.2MB/s] 
2990d213-b382-4b29-a180-1e57b490eda6.svs: 100%|██████████| 32/32 [00:49<00:00,  1.55s/it]
100%|██████████| 1.93G/1.93G [01:40<00:00, 19.2MB/s]
a62f9b6e-38eb-4a6b-9bc3-95038d7c14de.svs: 100%|██████████| 32/32 [00:46<00:00,  1.45s/it]
100%|██████████| 154M/154M [00:08<00:00, 19.0MB/s] 
b617630a-65cb-4e63-96fb-fa2bc41345af.svs: 100%|██████████| 32/32 [00:42<00:00,  1.31s/it]
100%|██████████| 666M/666M [00:34<00:00, 19.2MB/s] 
fca6bd6a-29f2-44cf-8351-31adeb67e816.svs: 100%|██████████| 32/32 [00:38<00:00,  1.21s/it]
100%|██████████| 1.65G/1.65G [01:25<00:00, 19.2MB/s]
f96a9f38-52c5-4736-8d00-c30e0b3cedf8.svs: 100%|██████████| 32/32 [00:43<00:00,  1.35s/it]
100%

[510/516] OK=509 FAIL=1


100%|██████████| 337M/337M [00:17<00:00, 19.1MB/s] 
22c2cc79-42bd-44dd-a305-34d0c841e105.svs: 100%|██████████| 32/32 [00:39<00:00,  1.25s/it]
100%|██████████| 761M/761M [00:39<00:00, 19.2MB/s] 
b3631f2e-098b-464a-8019-e8e16d682e61.svs: 100%|██████████| 32/32 [00:40<00:00,  1.28s/it]
100%|██████████| 464M/464M [00:24<00:00, 19.2MB/s] 
c6618582-3cd2-4861-8a50-d9e650f0bee6.svs: 100%|██████████| 32/32 [00:28<00:00,  1.12it/s]
100%|██████████| 1.11G/1.11G [00:57<00:00, 19.2MB/s]
1bf50fcd-9038-4deb-a221-5e68245eaa1e.svs: 100%|██████████| 32/32 [00:28<00:00,  1.12it/s]
100%|██████████| 1.26G/1.26G [01:05<00:00, 19.2MB/s]
54e55d74-b18d-47e2-ac66-fd8f9e1f6094.svs: 100%|██████████| 32/32 [00:28<00:00,  1.12it/s]
100%|██████████| 515M/515M [00:26<00:00, 19.1MB/s] 
eed858e8-ea64-40d8-a080-9551941ae1b5.svs: 100%|██████████| 32/32 [00:43<00:00,  1.34s/it]

DONE remaining. OK= 515 FAIL= 1


In [56]:
import pandas as pd

labels = pd.read_csv("multiomics_labels.tsv", sep="\t")

# normalize patient id column
if "patient_id" in labels.columns:
    patient_col = "patient_id"
elif "submitter_id" in labels.columns:
    patient_col = "submitter_id"
else:
    raise ValueError("No patient id column found")

labels_cases = set(labels[patient_col].astype(str).str.strip())

print("Cases in labels:", len(labels_cases))

Cases in labels: 831


In [58]:
from pathlib import Path

bags_dir = Path("bags")
bags_cases = set(p.stem for p in bags_dir.glob("*.h5"))

print("Cases with H5 files:", len(bags_cases))

Cases with H5 files: 830


In [59]:
missing_h5 = labels_cases - bags_cases
extra_h5   = bags_cases - labels_cases
matched    = labels_cases & bags_cases

print("Matched cases:", len(matched))
print("In labels but missing H5:", len(missing_h5))
print("In bags but not in labels:", len(extra_h5))

Matched cases: 830
In labels but missing H5: 1
In bags but not in labels: 0


In [60]:
if missing_h5:
    print("Examples missing H5:", list(missing_h5)[:10])

if extra_h5:
    print("Examples extra H5:", list(extra_h5)[:10])

Examples missing H5: ['TCGA-05-5420']


In [61]:
final_cases = sorted(matched)

pd.DataFrame({"case_id": final_cases}).to_csv(
    "cases_with_wsi_features.csv", index=False
)

print("Saved clean WSI case list:", len(final_cases))

Saved clean WSI case list: 830


In [62]:
print("Missing case:", list(missing_h5))

Missing case: ['TCGA-05-5420']


In [63]:
import pandas as pd

final_cases = sorted(matched)  # 830 cases with both label + H5
pd.DataFrame({"case_id": final_cases}).to_csv("cases_with_wsi_h5.csv", index=False)

print("Saved:", "cases_with_wsi_h5.csv", "with", len(final_cases), "cases")

Saved: cases_with_wsi_h5.csv with 830 cases


In [64]:
import os, glob, random
from pathlib import Path

import numpy as np
import pandas as pd
import h5py

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [65]:
labels_path = "multiomics_labels.tsv"
labels = pd.read_csv(labels_path, sep="\t")

# patient column
if "patient_id" in labels.columns:
    patient_col = "patient_id"
elif "submitter_id" in labels.columns:
    patient_col = "submitter_id"
else:
    raise ValueError(f"No patient id column found. Columns={labels.columns.tolist()}")

# label column
if "subtype_simple" in labels.columns:
    label_col = "subtype_simple"
elif "label" in labels.columns:
    label_col = "label"
else:
    raise ValueError(f"No label column found. Columns={labels.columns.tolist()}")

labels[patient_col] = labels[patient_col].astype(str).str.strip()
labels[label_col] = labels[label_col].astype(str).str.strip()

labels = labels[labels[label_col].isin(["LUAD", "LUSC"])].copy().reset_index(drop=True)

print("Label counts:")
print(labels[label_col].value_counts())
labels.head()

Label counts:
subtype_simple
LUAD    459
LUSC    372
Name: count, dtype: int64


,patient_id,subtype,subtype_simple
0,TCGA-MP-A4SV,TCGA-LUAD,LUAD
1,TCGA-55-8621,TCGA-LUAD,LUAD
2,TCGA-MN-A4N1,TCGA-LUAD,LUAD
3,TCGA-55-6986,TCGA-LUAD,LUAD
4,TCGA-86-6851,TCGA-LUAD,LUAD


In [66]:
bags_dir = Path("bags")
bags_cases = set(p.stem for p in bags_dir.glob("*.h5"))
labels_cases = set(labels[patient_col].tolist())

matched = sorted(labels_cases & bags_cases)
missing_h5 = sorted(labels_cases - bags_cases)
extra_h5   = sorted(bags_cases - labels_cases)

print("H5 cases:", len(bags_cases))
print("Labels cases:", len(labels_cases))
print("Matched:", len(matched))
print("Missing H5:", len(missing_h5))
print("Extra H5:", len(extra_h5))

if missing_h5:
    print("Missing example:", missing_h5[:5])

H5 cases: 830
Labels cases: 831
Matched: 830
Missing H5: 1
Extra H5: 0
Missing example: ['TCGA-05-5420']


In [67]:
label_map = {"LUAD": 0, "LUSC": 1}

labels_sub = labels[labels[patient_col].isin(matched)].copy()
labels_sub["case_id"] = labels_sub[patient_col]
labels_sub["y"] = labels_sub[label_col].map(label_map).astype(int)
labels_sub["h5_path"] = labels_sub["case_id"].apply(lambda x: str(bags_dir / f"{x}.h5"))

labels_sub = labels_sub[["case_id", "y", "h5_path"]].reset_index(drop=True)

print("Final usable cases:", len(labels_sub))
labels_sub.head()

Final usable cases: 830


,case_id,y,h5_path
0,TCGA-MP-A4SV,0,bags/TCGA-MP-A4SV.h5
1,TCGA-55-8621,0,bags/TCGA-55-8621.h5
2,TCGA-MN-A4N1,0,bags/TCGA-MN-A4N1.h5
3,TCGA-55-6986,0,bags/TCGA-55-6986.h5
4,TCGA-86-6851,0,bags/TCGA-86-6851.h5


In [68]:
sample_h5 = labels_sub.loc[0, "h5_path"]
with h5py.File(sample_h5, "r") as f:
    keys = list(f.keys())
    print("H5 keys:", keys)

H5 keys: ['coords', 'features']


In [69]:
def pick_feature_key(h5_path):
    with h5py.File(h5_path, "r") as f:
        keys = list(f.keys())
        # common choices
        for k in ["features", "patch_features", "embeddings", "feat", "x"]:
            if k in keys:
                return k
        # fallback: first 2D array
        for k in keys:
            obj = f[k]
            if hasattr(obj, "shape") and len(obj.shape) == 2:
                return k
    raise ValueError(f"Could not find a 2D feature dataset in {h5_path}")

FEATURE_KEY = pick_feature_key(sample_h5)
print("Using FEATURE_KEY =", FEATURE_KEY)

with h5py.File(sample_h5, "r") as f:
    X = f[FEATURE_KEY][:]
print("Sample feature shape:", X.shape)  # (num_patches, feature_dim)

Using FEATURE_KEY = features
Sample feature shape: (1987, 1024)


In [70]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    labels_sub, test_size=0.2, random_state=SEED, stratify=labels_sub["y"]
)

train_df, val_df = train_test_split(
    train_df, test_size=0.2, random_state=SEED, stratify=train_df["y"]
)

print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))
print("Train label balance:\n", train_df["y"].value_counts(normalize=True))

Train: 531 Val: 133 Test: 166
Train label balance:
 y
0    0.551789
1    0.448211
Name: proportion, dtype: float64


In [71]:
class WSIBagDataset(Dataset):
    def __init__(self, df, feature_key, max_patches=2048, shuffle_patches=True):
        self.df = df.reset_index(drop=True)
        self.feature_key = feature_key
        self.max_patches = max_patches
        self.shuffle_patches = shuffle_patches

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        h5_path = row["h5_path"]
        y = int(row["y"])

        with h5py.File(h5_path, "r") as f:
            feats = f[self.feature_key][:]
        
        # feats: (N, D)
        N = feats.shape[0]
        if self.max_patches is not None and N > self.max_patches:
            if self.shuffle_patches:
                sel = np.random.choice(N, size=self.max_patches, replace=False)
            else:
                sel = np.arange(self.max_patches)
            feats = feats[sel]

        feats = torch.from_numpy(feats).float()  # (n, d)
        y = torch.tensor(y, dtype=torch.long)
        return feats, y

In [72]:
def mil_collate(batch):
    bags, labels = zip(*batch)  # bags is list of (n_i, d)
    labels = torch.stack(labels)
    return list(bags), labels

In [73]:
train_ds = WSIBagDataset(train_df, FEATURE_KEY, max_patches=2048)
val_ds   = WSIBagDataset(val_df, FEATURE_KEY, max_patches=2048)
test_ds  = WSIBagDataset(test_df, FEATURE_KEY, max_patches=2048)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=mil_collate, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=1, shuffle=False, collate_fn=mil_collate, num_workers=0)
test_loader  = DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=mil_collate, num_workers=0)

# sanity check one batch
bags, y = next(iter(train_loader))
print(type(bags), len(bags), bags[0].shape, y)

<class 'list'> 1 torch.Size([2000, 1024]) tensor([1])


In [74]:
class GatedAttentionMIL(nn.Module):
    def __init__(self, in_dim, hidden_dim=256, num_classes=2, dropout=0.25):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        self.attn_V = nn.Sequential(nn.Linear(hidden_dim, hidden_dim), nn.Tanh())
        self.attn_U = nn.Sequential(nn.Linear(hidden_dim, hidden_dim), nn.Sigmoid())
        self.attn_w = nn.Linear(hidden_dim, 1)

        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, bag):  # bag: (n, d)
        H = self.fc(bag)  # (n, h)
        A = self.attn_w(self.attn_V(H) * self.attn_U(H))  # (n, 1)
        A = torch.softmax(A, dim=0)  # (n, 1)

        M = torch.sum(A * H, dim=0)  # (h,)
        logits = self.classifier(M).unsqueeze(0)  # (1, C)
        return logits, A.squeeze(1)  # logits, attn weights (n,)

In [75]:
# infer feature dim from one H5
with h5py.File(labels_sub.loc[0, "h5_path"], "r") as f:
    D = f[FEATURE_KEY].shape[1]

print("Feature dim D =", D)

model = GatedAttentionMIL(in_dim=D, hidden_dim=256, num_classes=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

print(model)

Feature dim D = 1024
GatedAttentionMIL(
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
  )
  (attn_V): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): Tanh()
  )
  (attn_U): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): Sigmoid()
  )
  (attn_w): Linear(in_features=256, out_features=1, bias=True)
  (classifier): Linear(in_features=256, out_features=2, bias=True)
)
